Point of this is to convert .xml files into json files for entry point into CDE modeller.

Directories

In [2]:
database_path = 'Hackathon_Data/cde_database/full_database/'

Import libraries

In [2]:
import xml.etree.ElementTree as ET
import os
#from bs4 import BeautifulSoup
#import json

In [5]:


for filename in os.listdir(database_path):
    if not filename.endswith('.xml'):
        continue
    fullname = os.path.join(database_path, filename)
    tree = ET.parse(fullname)
    #tree = BeautifulSoup(open(fullname),'xml')


In [3]:
tree = ET.parse('Hackathon_Data/cde_database/full_database/xml_cde_2020102112645_10.xml')

root = tree.getroot()

# Get all public_ids
cde_public_ids = []
for public_id in root.iter('PUBLICID'):
    cde_public_ids.append(public_id.text)

In [5]:
import json
import xmltodict
with open("Hackathon_Data/cde_database/full_database/xml_cde_2020102112645_10.xml") as xml_file:
    data_dict = xmltodict.parse(xml_file.read())

json_data = json.loads(data_dict)

In [10]:
data_element_list = json.loads(json_data)['DataElementsList']['DataElement']

In [98]:
correct_keys = {'LONGNAME':'long_name',
                'PREFERREDNAME':'preferred_name',
                'PREFERREDDEFINITION':'preferred_definition',
                'DATAELEMENTCONCEPT':'concept',
                'PublicId':'public_id',
                'PreferredDefinition':'preferred_definition',
                'LongName':'long_name',
                'Datatype' : 'data_type',
                'UnitOfMeasure': 'unit_of_measure',
                'MaximumValue':'maximum_value',
                'MinimumValue':'minimum_value',
                'PermissibleValues':'permissible_values',
                'VALIDVALUE':'valid_value',
                'VMPUBLICID' : 'public_id'
                }

def keyparse(key_string):
    return correct_keys[key_string]

In [93]:
new_dict = {}
concept_dict = {}

list_of_keys_main = ['LONGNAME','PREFERREDNAME','PREFERREDDEFINITION']
list_of_keys_concept = ['PublicId','PreferredDefinition']
list_of_keys_value = ['PublicId','PreferredDefinition','LongName','Datatype','UnitOfMeasure','MaximumValue','MinimumValue','PermissibleValues']
list_of_keys_permissible_values = ['VALIDVALUE','VMPUBLICID']
for data_element in data_element_list:
    for (key, value) in data_element.items():
        if key == 'PUBLICID':
            new_key = value
        else: 
            if key in list_of_keys_main:
                new_dict[new_key] = {}
                new_dict[new_key][key] = value # to create dict of all elements
            elif key == 'DATAELEMENTCONCEPT':
                concept_dict = {key_concept: data_element[key][key_concept] for key_concept in list_of_keys_concept}
                new_dict[new_key][key] = concept_dict
            elif key == 'VALUEDOMAIN':
                value_dict = {}
                value_dict = {key_value: data_element[key][key_value] for key_value in list_of_keys_value}

                if value_dict['PermissibleValues'] != None:
                    perm_value_list = value_dict['PermissibleValues']['PermissibleValues_ITEM']
                    permissible_values = []

                    if type(perm_value_list) == list:
                        for value in perm_value_list:
                            permissible_value = {}
                            permissible_value = {key_perm_value: value[key_perm_value] for key_perm_value in list_of_keys_permissible_values}
                            permissible_values.append(permissible_value)
                    else:
                        permissible_value = {}
                        permissible_value = {key_perm_value: perm_value_list[key_perm_value] for key_perm_value in list_of_keys_permissible_values}
                        permissible_values.append(permissible_value)                                   
                        
                    value_dict['PermissibleValues'] = permissible_values

                new_dict[new_key][key] = value_dict               
                
print(new_dict['3679498']['VALUEDOMAIN'].keys())

dict_keys(['PublicId', 'PreferredDefinition', 'LongName', 'Datatype', 'UnitOfMeasure', 'MaximumValue', 'MinimumValue', 'PermissibleValues'])
